In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

# Step 1: Read the Excel file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM.xlsx'
df = pd.read_excel(file_path)

# Step 2: Load the pre-trained model
model = SentenceTransformer('/data1/dxw_data/llm/paraphrase-multilingual-MiniLM-L12-v2')

# Step 3: Define the category groups
categories = [
    "Product Guide", "Product Review", "Outfit Recommendations",
    "Educational/Informational", "Popular Science", "Medical Health", "Education and Campus", "Workplace/Interpersonal Relationships", "Technology",
    "Plot Type", "Movie Commentary", "Film/Variety Editing", "Short Skits", "Animation/Anime",
    "Leisure and Comedy", "Creative Editing/Dubbing", "Satirical Parody", "Art Creation", "Street Interviews", "Landscape Photography", "Cute Pets",
    "Life Sharing", "Food", "Vlog/Insights Sharing", "Travel", "Fitness and Beauty", "Skill Sharing", "Home Life",
    "Appearance", "Dance", "Music",
    "Advertising", "Local Culture and Tourism", "Public Service Advertising",
    "Celebrity Entertainment",
    "News and Current Events", "Social and Political News",
    "Games",
    "Sports",
    "Automobiles",
    "Finance",
    "Others"
]

# Step 4: Calculate embeddings for category keywords
category_embeddings = model.encode(categories)

# Function to find the closest category for a given description keyword
def find_closest_category(keyword):
    keyword_embedding = model.encode(keyword)
    similarities = util.pytorch_cos_sim(keyword_embedding, category_embeddings)
    closest_idx = similarities.argmax()
    return categories[closest_idx]

# Step 5: Process each description and replace keywords
def replace_keywords(description):
    if isinstance(description, str):
        keywords = [kw.strip() for kw in description.split(';')]
        replaced_keywords = [find_closest_category(kw) for kw in keywords]
        return '; '.join(replaced_keywords)
    return description  # Return the original value if it's not a string

# Apply the replacement to each row with progress indication
tqdm.pandas()
df['Modified_Description'] = df['Description'].progress_apply(replace_keywords)

# Step 6: Save the modified DataFrame back to the Excel file
df.to_excel(file_path, index=False)

print("Descriptions have been successfully replaced and saved to the Excel file.")


100%|██████████| 9698/9698 [12:27<00:00, 12.97it/s]  


Descriptions have been successfully replaced and saved to the Excel file.


In [6]:
import pandas as pd

# Read DatafromLLM.xlsx
data_llm = pd.read_excel('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM.xlsx')

# Step 1: Filter rows where Time is greater than 2
data_llm_valid = data_llm[data_llm['Time'] > 2]

# Step 2: Group by UserID and Video, then combine Modified_Description into a nested list structure
def combine_descriptions(description_series):
    # Split each description into a list of categories
    lists_of_categories = [desc.split(';') for desc in description_series]
    # Remove extra spaces from categories and return as nested list
    return [list(map(lambda x: x.strip(), category_list)) for category_list in lists_of_categories]

grouped_llm = data_llm_valid.groupby(['UserID', 'Video']).agg({
    'Modified_Description': combine_descriptions
}).reset_index()

# Rename column for clarity
grouped_llm.columns = ['UserID', 'Video', 'Main_Category_list']

# Save the result to a new CSV
grouped_llm.to_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories_combined.csv', index=False)

print("Data processing completed and CSV file saved.")


Data processing completed and CSV file saved.


In [15]:
# 大类匹配

import pandas as pd
import ast
import re

# Define the category groups
categories = [
    ["Product Guide", "Product Review", "Outfit Recommendations"],
    ["Educational/Informational", "Popular Science", "Medical Health", "Education and Campus", "Workplace/Interpersonal Relationships", "Technology"],
    ["Plot Type", "Movie Commentary", "Film/Variety Editing", "Short Skits", "Animation/Anime"],
    ["Leisure and Comedy", "Creative Editing/Dubbing", "Satirical Parody", "Art Creation", "Street Interviews", "Landscape Photography", "Cute Pets"],
    ["Life Sharing", "Food", "Vlog/Insights Sharing", "Travel", "Fitness and Beauty", "Skill Sharing", "Home Life"],
    ["Appearance", "Dance", "Music"],
    ["Advertising", "Local Culture and Tourism", "Public Service Advertising"],
    ["Celebrity Entertainment"],
    ["News and Current Events", "Social and Political News"],
    ["Games"],
    ["Sports"],
    ["Automobiles"],
    ["Finance"],
    ["Others"]
]

# Create a dictionary to map each category to its group index
category_to_group = {category: index for index, group in enumerate(categories) for category in group}

# Read the CSV files
llm_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories_combined.csv')
ra_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromRA_Main_Categoriess_combined.csv')

# Function to preprocess and evaluate category strings safely
def preprocess_and_eval(category_string):
    cleaned_string = re.sub(r'\s+', ' ', category_string.strip())
    if not (cleaned_string.startswith("[") and cleaned_string.endswith("]")):
        cleaned_string = f"[{cleaned_string}]"
    try:
        return ast.literal_eval(cleaned_string)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {category_string}")
        return []

# Function to map categories to their group indices
def map_categories_to_groups(category_list, category_to_group):
    return [category_to_group.get(cat, -1) for cat in category_list]

# Function to find the maximum length of the matching subsequence
def find_max_matching_sequence(llm_list, ra_list, category_to_group):
    llm_groups = [map_categories_to_groups(sublist, category_to_group) for sublist in llm_list]
    ra_groups = [map_categories_to_groups(sublist, category_to_group) for sublist in ra_list]

    llm_elements = [el for sublist in llm_groups for el in sublist]
    ra_elements = [el for sublist in ra_groups for el in sublist]

    len_llm = len(llm_elements)
    len_ra = len(ra_elements)

    # Create a 2D array to store the lengths of longest common subsequence
    dp = [[0] * (len_llm + 1) for _ in range(len_ra + 1)]

    for i in range(1, len_ra + 1):
        for j in range(1, len_llm + 1):
            if ra_elements[i - 1] == llm_elements[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    return dp[len_ra][len_llm]

# Prepare a list to store the results
results = []

# Iterate over the rows in ra_df
for _, ra_row in ra_df.iterrows():
    user_id = ra_row['UserID']
    video_id = ra_row['Video']

    # Find the corresponding LLM row
    llm_row = llm_df[(llm_df['UserID'] == user_id) & (llm_df['Video'] == video_id)]

    if not llm_row.empty:
        llm_categories = llm_row.iloc[0]['Main_Category_list']
        ra_categories = ra_row['Main_Category_list']

        # Preprocess and evaluate category strings
        llm_list = preprocess_and_eval(llm_categories)
        ra_list = preprocess_and_eval(ra_categories)

        # Calculate all_num and sequence_num
        all_num = len(ra_list)  # Use the length of the RA list
        sequence_num = find_max_matching_sequence(llm_list, ra_list, category_to_group)

        # Append the results
        results.append((user_id, video_id, all_num, sequence_num))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['UserID', 'Video', 'all_num', 'sequence_num'])


# Save the results to a new CSV file
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_combined.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Error parsing string: [['Others'], ['Life Sharing'], ['Life Sharing'], ['Life Sharing'], nan, ['Life Sharing'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Plot Type'], ['Leisure and Comedy'], nan, ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Life Sharing'], ['Others'], ['Leisure and Comedy']]
Error parsing string: [['Leisure and Comedy'], ['News and Current Events'], ['News and Current Events'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Others'], nan, nan, ['Life Sharing'], ['Plot Type'], nan, ['Leisure and Comedy'], ['Life Sharing'], ['Appearance'], ['Life Sharing'], nan, ['Life Sharing'], ['Sports'], ['Life Sharing'], ['News and Current Events'], nan, ['Sports'], ['Sports'], ['Sports'], ['Sports']]
Error parsing string: [['Life Sharing'], ['Appearance'], ['Leisure and Comedy'], ['Appearance'], ['Appearance'], ['Leisure and Comedy'], nan, ['Appearance'], ['Appearance']]


In [ ]:
# 大类匹配

import pandas as pd
import ast
import re

# Define the category groups
categories = [
    ["Product Guide", "Product Review", "Outfit Recommendations"],
    ["Educational/Informational", "Popular Science", "Medical Health", "Education and Campus", "Workplace/Interpersonal Relationships", "Technology"],
    ["Plot Type", "Movie Commentary", "Film/Variety Editing", "Short Skits", "Animation/Anime"],
    ["Leisure and Comedy", "Creative Editing/Dubbing", "Satirical Parody", "Art Creation", "Street Interviews", "Landscape Photography", "Cute Pets"],
    ["Life Sharing", "Food", "Vlog/Insights Sharing", "Travel", "Fitness and Beauty", "Skill Sharing", "Home Life"],
    ["Appearance", "Dance", "Music"],
    ["Advertising", "Local Culture and Tourism", "Public Service Advertising"],
    ["Celebrity Entertainment"],
    ["News and Current Events", "Social and Political News"],
    ["Games"],
    ["Sports"],
    ["Automobiles"],
    ["Finance"],
    ["Others"]
]

# Create a dictionary to map each category to its group index
category_to_group = {category: index for index, group in enumerate(categories) for category in group}

# Read the CSV files
llm_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories_combined.csv')
ra_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromRA_Main_Categoriess_combined.csv')

# Function to preprocess and evaluate category strings safely
def preprocess_and_eval(category_string):
    # Remove newlines and extra spaces
    cleaned_string = re.sub(r'\s+', ' ', category_string.strip())
    # Ensure it's a list format
    if not (cleaned_string.startswith("[") and cleaned_string.endswith("]")):
        cleaned_string = f"[{cleaned_string}]"
    try:
        return ast.literal_eval(cleaned_string)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {category_string}")
        return []

# Function to map categories to their group indices
def map_categories_to_groups(category_list, category_to_group):
    return [category_to_group.get(cat, -1) for cat in category_list]

# Function to calculate sequence match with the shorter list matching against the longer list
def calculate_sequence_match(llm_list, ra_list, category_to_group):
    llm_groups = [map_categories_to_groups(sublist, category_to_group) for sublist in llm_list]
    ra_groups = [map_categories_to_groups(sublist, category_to_group) for sublist in ra_list]

    sequence_num = 0

    for ra_sublist in ra_groups:
        ra_group = ra_sublist[0] if ra_sublist else -1
        for llm_sublist in llm_groups:
            if ra_group in llm_sublist:
                sequence_num += 1
                break

    return sequence_num

# Prepare a list to store the results
results = []

# Iterate over the rows in ra_df
for _, ra_row in ra_df.iterrows():
    user_id = ra_row['UserID']
    video_id = ra_row['Video']

    # Find the corresponding LLM row
    llm_row = llm_df[(llm_df['UserID'] == user_id) & (llm_df['Video'] == video_id)]

    if not llm_row.empty:
        llm_categories = llm_row.iloc[0]['Main_Category_list']
        ra_categories = ra_row['Main_Category_list']

        # Preprocess and evaluate category strings
        llm_list = preprocess_and_eval(llm_categories)
        ra_list = preprocess_and_eval(ra_categories)

        # Calculate all_num and sequence_num
        all_num = len(ra_list)  # Use the length of the RA list
        sequence_num = calculate_sequence_match(llm_list, ra_list, category_to_group)

        # Append the results
        results.append((user_id, video_id, all_num, sequence_num))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['UserID', 'Video', 'all_num', 'sequence_num'])

# Save the results to a new CSV file
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_combined.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


In [16]:
import pandas as pd
import ast
import re

# Read the CSV files
llm_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories_combined.csv')
ra_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromRA_Main_Categoriess_combined.csv')


# Function to preprocess and evaluate category strings safely
def preprocess_and_eval(category_string):
    cleaned_string = re.sub(r'\s+', ' ', category_string.strip())
    if not (cleaned_string.startswith("[") and cleaned_string.endswith("]")):
        cleaned_string = f"[{cleaned_string}]"
    try:
        return ast.literal_eval(cleaned_string)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {category_string}")
        return []

# Function to find the maximum length of the matching subsequence
def find_max_matching_sequence(llm_list, ra_list):
    llm_elements = [el for sublist in llm_list for el in sublist]
    ra_elements = [el for sublist in ra_list for el in sublist]

    len_llm = len(llm_elements)
    len_ra = len(ra_elements)

    # Create a 2D array to store the lengths of longest common subsequence
    dp = [[0] * (len_llm + 1) for _ in range(len_ra + 1)]

    for i in range(1, len_ra + 1):
        for j in range(1, len_llm + 1):
            if ra_elements[i - 1] == llm_elements[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    return dp[len_ra][len_llm]

# Prepare a list to store the results
results = []

# Iterate over the rows in ra_df
for _, ra_row in ra_df.iterrows():
    user_id = ra_row['UserID']
    video_id = ra_row['Video']

    # Find the corresponding LLM row
    llm_row = llm_df[(llm_df['UserID'] == user_id) & (llm_df['Video'] == video_id)]

    if not llm_row.empty:
        llm_categories = llm_row.iloc[0]['Main_Category_list']
        ra_categories = ra_row['Main_Category_list']

        # Preprocess and evaluate category strings
        llm_list = preprocess_and_eval(llm_categories)
        ra_list = preprocess_and_eval(ra_categories)

        # Calculate all_num and sequence_num
        all_num = len(ra_list)  # Use the length of the RA list
        sequence_num = find_max_matching_sequence(llm_list, ra_list)

        # Append the results
        results.append((user_id, video_id, all_num, sequence_num))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['UserID', 'Video', 'all_num', 'sequence_num'])


# Save the results to a new CSV file
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_combined2.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Error parsing string: [['Others'], ['Life Sharing'], ['Life Sharing'], ['Life Sharing'], nan, ['Life Sharing'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Plot Type'], ['Leisure and Comedy'], nan, ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Life Sharing'], ['Others'], ['Leisure and Comedy']]
Error parsing string: [['Leisure and Comedy'], ['News and Current Events'], ['News and Current Events'], ['Leisure and Comedy'], ['Leisure and Comedy'], ['Others'], nan, nan, ['Life Sharing'], ['Plot Type'], nan, ['Leisure and Comedy'], ['Life Sharing'], ['Appearance'], ['Life Sharing'], nan, ['Life Sharing'], ['Sports'], ['Life Sharing'], ['News and Current Events'], nan, ['Sports'], ['Sports'], ['Sports'], ['Sports']]
Error parsing string: [['Life Sharing'], ['Appearance'], ['Leisure and Comedy'], ['Appearance'], ['Appearance'], ['Leisure and Comedy'], nan, ['Appearance'], ['Appearance']]


In [17]:
import pandas as pd

# Load the CSV file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_combined.csv'
data = pd.read_csv(file_path)

# Calculate totals for each UserID and Video combination
user_video_totals = data.groupby(['UserID', 'Video']).agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy with condition for each UserID and Video
user_video_totals['accuracy'] = user_video_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate totals and average match rate for each UserID
user_totals = data.groupby('UserID').agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy for each UserID with condition
user_totals['accuracy'] = user_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate average match rate for each UserID
average_match_rate = user_video_totals.groupby('UserID')['accuracy'].mean()

# Write results to a text file
output_file_path = 'accuracy_results_multimatch-maincategory.txt'
with open(output_file_path, 'w') as f:
    f.write("Totals and Accuracy for Each UserID and Video Combination:\n")
    f.write(user_video_totals.to_string())
    f.write("\n\nTotals and Accuracy for Each UserID:\n")
    f.write(user_totals.to_string())
    f.write("\n\nAverage Match Rate for Each UserID:\n")
    f.write(average_match_rate.to_string())

print(f"Results have been saved to {output_file_path}")


Results have been saved to accuracy_results_multimatch-maincategory.txt


/tmp/ipykernel_738239/3939665530.py:12: RuntimeWarning: invalid value encountered in scalar divide
  lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],


In [18]:
import pandas as pd

# Load the CSV file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_combined2.csv'
data = pd.read_csv(file_path)

# Calculate totals for each UserID and Video combination
user_video_totals = data.groupby(['UserID', 'Video']).agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy with condition for each UserID and Video
user_video_totals['accuracy'] = user_video_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate totals and average match rate for each UserID
user_totals = data.groupby('UserID').agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy for each UserID with condition
user_totals['accuracy'] = user_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate average match rate for each UserID
average_match_rate = user_video_totals.groupby('UserID')['accuracy'].mean()

# Write results to a text file
output_file_path = 'accuracy_results_multimatch-subcategory.txt'
with open(output_file_path, 'w') as f:
    f.write("Totals and Accuracy for Each UserID and Video Combination:\n")
    f.write(user_video_totals.to_string())
    f.write("\n\nTotals and Accuracy for Each UserID:\n")
    f.write(user_totals.to_string())
    f.write("\n\nAverage Match Rate for Each UserID:\n")
    f.write(average_match_rate.to_string())

print(f"Results have been saved to {output_file_path}")


Results have been saved to accuracy_results_multimatch-subcategory.txt


/tmp/ipykernel_738239/1386846258.py:12: RuntimeWarning: invalid value encountered in scalar divide
  lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
